In [9]:
import pandas as pd
# To install, pip install tabula-py in your env
import tabula

In [10]:
tabula.convert_into(r"resources\2020_data.pdf", r"uncleaned_csv\adult_access2020.csv", output_format="csv", pages='34', stream = True)

In [11]:
# Read in csv
adult_access_df = pd.read_csv(r"uncleaned_csv\adult_access2020.csv")
adult_access_df
adult_access_df = adult_access_df.dropna(how='all')

In [12]:
# Extract second half of ami_df to make second df to be joined back in
adult_access_df_2 = adult_access_df[["Rank", "State", "%.1", "#.1"]]
adult_access_df_2 = adult_access_df_2.rename(columns={"%.1" : "%", "#.1": "#"})
adult_access_df_2

,Rank,State,%,#
0,27.0,District of Columbia,55.4,"68,000"
2,28.0,North Dakota,55.6,"54,000"
3,29.0,South Dakota,55.6,"61,000"
4,30.0,Connecticut,55.7,"277,000"
5,31.0,Illinois,55.7,"847,000"
6,32.0,Maryland,55.9,"446,000"
8,33.0,Kansas,56.8,"238,000"
9,34.0,Idaho,57.1,"188,000"
10,35.0,South Carolina,57.4,"397,000"
11,36.0,Montana,58.3,"92,000"


In [13]:
adult_access_df = adult_access_df.drop(columns={"Rank", "State", "%.1", "#.1"})

In [14]:
adult_access_df['Rank'], adult_access_df['State'] = adult_access_df['Rank State'].str.split(' ', 1).str

adult_access_df.drop(columns='Rank State')

C:\Users\noaht\AppData\Local\Temp/ipykernel_4620/2023966433.py:1: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
  adult_access_df['Rank'], adult_access_df['State'] = adult_access_df['Rank State'].str.split(' ', 1).str


,%,#,Rank,State
0,40.7,"42,000",1,Vermont
2,44.6,"89,000",2,New Hampshire
3,45.7,"73,000",3,Rhode Island
4,47.3,"214,000",4,Iowa
5,47.5,"64,000",5,Delaware
6,48.7,"407,000",6,Colorado
8,49.5,"360,000",7,Minnesota
9,49.6,"99,000",8,Maine
10,50.9,"485,000",9,Missouri
11,51.1,"713,000",10,Michigan


In [15]:
adult_access_df = adult_access_df[['Rank', 'State', '%', '#']]
adult_access_df = adult_access_df.append(adult_access_df_2)

In [16]:
adult_access_df = adult_access_df.reset_index(drop=True)

In [17]:
adult_access_df

,Rank,State,%,#
0,1,Vermont,40.7,"42,000"
1,2,New Hampshire,44.6,"89,000"
2,3,Rhode Island,45.7,"73,000"
3,4,Iowa,47.3,"214,000"
4,5,Delaware,47.5,"64,000"
5,6,Colorado,48.7,"407,000"
6,7,Minnesota,49.5,"360,000"
7,8,Maine,49.6,"99,000"
8,9,Missouri,50.9,"485,000"
9,10,Michigan,51.1,"713,000"


In [18]:
adult_access_df.loc[15, "Rank"]=16
adult_access_df.loc[15, "State"]='Nebraska'
adult_access_df.loc[41, "#"]=829000

In [21]:
adult_access_df = adult_access_df.add_suffix('_adult_care_access_2020')

In [22]:
adult_access_df.to_csv(r'resources\adult_access2020.csv')